# **Modelling and Evaluation**

## Objectives

**Perform Business requirement 2 user story tasks: model selection, pipeline creation, hyperparameter tuning, model evaluation.**
* Create initial data cleaning and engineering pipeline using information from previous notebooks.
* Create initial modelling and evaluation pipeline using information from previous notebooks.
* Find best model candidate.
* Optimise chosen model through tuning and feature selection using feature importance. 
* Evaluate the model performance using residual plots and performance metrics.
* Successfully achieve $R^2 \ge 0.75$ for the final model, to satisfy the client's success criteria, and thereby satisfy business requirement 2.


## Inputs
* house prices dataset: outputs/datasets/collection/house_prices.csv.
* Information regarding the steps to include in the various pipelines, as indicated in the conclusion sections of the data cleaning and feature engineering notebooks.
* Outlier indices list: outputs/ml/outlier_indices.pkl

## Outputs

---

## Change working directory

Working directory changed to its parent folder.

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
os.getcwd()

---

## Load house price dataset

In [ ]:
import pandas as pd

house_prices_df = pd.read_csv(filepath_or_buffer='outputs/datasets/collection/house_prices.csv')

---

## Removing know outliers from whole dataset

Loading outlier indices list

In [ ]:
import joblib
outlier_indices = joblib.load('outputs/ml/outlier_indices.pkl')
outlier_indices

Removing the instances

In [ ]:
house_prices_df.drop(labels=outlier_indices, inplace=True)

---

## Create data cleaning and feature engineering pipeline

In [ ]:
import numpy as np
import src.transformers_and_functions as tf
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from feature_engine.selection import SmartCorrelatedSelection, DropFeatures
from sklearn.tree import DecisionTreeRegressor



In [ ]:
def data_cleaning_and_feature_engineering():
    """
    Constructs and returns data cleaning and feature engineering pipeline.
    """
    # variables for defining pipeline
    bsmt_fin_type1_cat = np.array(list(reversed(['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'None'])))
    bsmt_exposure_cat = np.array(['None', 'No', 'Mn', 'Av', 'Gd'])
    garage_finish_cat = np.array(['None', 'Unf', 'RFn', 'Fin'])
    kitchen_quality_cat = np.array(['Po', 'Fa', 'TA', 'Gd', 'Ex'])
    categories = [bsmt_exposure_cat, bsmt_fin_type1_cat, garage_finish_cat, kitchen_quality_cat]

    estimator = DecisionTreeRegressor(min_samples_split=10, min_samples_leaf=5, random_state=30)
    smart_variables = ['GrLivArea', 'GarageYrBlt', 'TotalBsmtSF', 'GarageArea', 'BsmtFinSF1',
                       'EnclosedPorchSF', 'GarageFinish', 'KitchenQual', 'YearBuilt', 'YearRemodAdd',
                       'OverallQual', '1stFlrSF', '2ndFlrSF']

    pipeline = Pipeline([
                        # Data cleaning:
                        # Missing value imputation:
                        ('IndependentKNNImputer', tf.IndependentKNNImputer()),
                        ('EqualFrequencyImputer', tf.EqualFrequencyImputer()),
                        #feature engineering:
                        # encoding:
                        ('OrdinalEncoder', OrdinalEncoder(categories=categories, dtype='int64')),
                        # feature number reduction
                        ('CompositeSelectKBest'), tf.CompositeSelectKBest(),
                        ('SmartCorrelatedSelection', SmartCorrelatedSelection(variables=smart_variables, method='spearman',
                                                                              threshold=0.8, selection_method='model_performance',
                                                                              estimator=estimator, scoring='r2', cv=5)),
                        ('DropFeatures', DropFeatures('EnclosedPorchSF')),
                        #feature scaling:
                        ('CompositeNormaliSer', tf.CompositeNormaliser())])
    return pipeline

In [ ]:
data_cleaning_and_feature_engineering_pipeline = data_cleaning_and_feature_engineering()

## Split dataset

In [ ]:
from sklearn.model_selection import train_test_split

(train_set_df, test_set_df) = train_test_split(house_prices_df, test_size=0.25, random_state=30)

---

## Scale target

In [ ]:
min_max_scaler = MinMaxScaler()
min_max_scaler.set_output(transform='pandas')
min_max_scaler.fit(train_set_df[['SalePrice']])

Transform train set target

In [ ]:
transformed_df = min_max_scaler.transform(train_set_df[['SalePrice']])
train_set_df[['SalePrice']] = transformed_df

Transform test set target

In [ ]:
transformed_df = min_max_scaler.transform(test_set_df[['SalePrice']])
test_set_df[['SalePrice']] = transformed_df

---